In [3]:
%load_ext autoreload
%autoreload 2

import os
os.environ['CUDA_VISIBLE_DEVICES']=''

import sys;
sys.path.append('..')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import json
wiki = json.load(open('/home/kuratov/data/ThePile/Wikipedia/wikipedia-en-0.json', 'r'))

In [9]:
{
    'title': '',
    'text': '',
}

{'title': '', 'text': ''}

In [10]:
# articles to skip:
# List of ...
# {title} may refer to: (disambiguation pages)
# wiki key words to clean
# \nCategory:
# \n\nSee also\n\n
# \n\nReferences\n\n

In [5]:
import re
category_regexp = re.compile(r'\ACategory:[\S ]+\Z')
keywords_regexp = re.compile(r'\n(References|See also|External Links)\s?\n')
title_regexp = re.compile(r'\A[\S ]+?\\n\\n')
list_of_regexp = re.compile(r'\AList of')

def clean_article(text):
    lines = []
    title = ''
    for i, line in enumerate(text.split('\n')):
        if i == 0:
            title = line
            continue
        if category_regexp.match(line):
            continue
        lines += [line]
    text = '\n'.join(lines)
    m = re.search(keywords_regexp, text)
    if m:
        text = text[:m.span()[0]]
    return title, text.strip()

In [6]:
# preprocess wiki
from tqdm import tqdm as tqdm
import json

path = '/home/kuratov/data/ThePile/Wikipedia/wikipedia-en-{}.json'

total_articles = 0
filtered_articles = 0

articles = []

for i in tqdm(range(0, 10)):
    wiki = json.load(open(path.format(i), 'r'))
    for text in wiki:
        total_articles +=1
        title, text = clean_article(text)
        if re.search(list_of_regexp, title):
            filtered_articles += 1
            continue
        if f'{title} may refer to:' in text:
            filtered_articles += 1
            continue
        articles += [{'title': title, 'text': text}]
print(f'{filtered_articles}/{total_articles}')


100%|██████████| 10/10 [05:54<00:00, 35.46s/it]

279906/6033151


In [7]:
print(f'{filtered_articles}/{len(articles)}/{total_articles}')

279906/5753245/6033151


In [8]:
# save shards

from pathlib import Path

shards_path = '/home/kuratov/data/ThePile/Wikipedia/preprocessed_shards_train'

if not Path(shards_path).exists():
    Path(shards_path).mkdir(parents=True)

import random, math
random.shuffle(articles)
n_shards = 81  # (we possibly have to take n_gpus shards for validation, so we need more shards) add +1 for validation

shard_size = math.ceil(len(articles) / n_shards)

for i, start in enumerate(tqdm(range(0, len(articles), shard_size))):
    with (Path(shards_path) / f'shard_{i:02d}.jsonl').open('w', encoding='utf8') as fout:
        print(start, start + shard_size)
        for j in range(start, min(len(articles), start+shard_size)):
            jsoned = json.dumps(articles[j])
            try:
                tmp = json.loads(jsoned)
            except json.JSONDecodeError as e:
                print(f'skipping article {j} in shard {i} cause of JSONDecodeError')
                continue
            fout.write(jsoned+'\n')


  0%|          | 0/81 [00:00<?, ?it/s]

0 71028



  1%|          | 1/81 [00:02<03:47,  2.85s/it]

71028 142056



  2%|▏         | 2/81 [00:05<03:44,  2.84s/it]

...


 98%|█████████▊| 79/81 [03:26<00:05,  2.60s/it]

5611212 5682240



 99%|█████████▉| 80/81 [03:29<00:02,  2.56s/it]

5682240 5753268



100%|██████████| 81/81 [03:31<00:00,  2.61s/it]


In [23]:
# save shards, with valid shard 0.5 of train_shard size

from pathlib import Path

shards_path = '/home/kuratov/data/ThePile/Wikipedia/preprocessed_shards_train'

if not Path(shards_path).exists():
    Path(shards_path).mkdir(parents=True)

import random, math
random.shuffle(articles)
n_train_shards = 80  # (we possibly have to take n_gpus shards for validation, so we need more shards)
n_valid_shards = 1 #  add +1 for validation, only 1 works currently
valid_shard_size = 0.5 # half of the train shard

shard_size = math.ceil(len(articles) / (n_train_shards + n_valid_shards))
shard_size = math.ceil((len(articles) - shard_size * valid_shard_size) / n_train_shards)

for i, start in enumerate(tqdm(range(0, len(articles), shard_size))):
    with (Path(shards_path) / f'shard_{i:02d}.jsonl').open('w', encoding='utf8') as fout:
        print(start, start + shard_size)
        for j in range(start, min(len(articles), start+shard_size)):
            jsoned = json.dumps(articles[j])
            try:
                tmp = json.loads(jsoned)
            except json.JSONDecodeError as e:
                print(f'skipping article {j} in shard {i} cause of JSONDecodeError')
                continue
            fout.write(jsoned+'\n')


  0%|          | 0/81 [00:00<?, ?it/s]

0 71472



  1%|          | 1/81 [00:02<03:25,  2.57s/it]

71472 142944



  2%|▏         | 2/81 [00:05<03:21,  2.56s/it]

...


 98%|█████████▊| 79/81 [03:18<00:05,  2.58s/it]

5646288 5717760



 99%|█████████▉| 80/81 [03:20<00:02,  2.67s/it]

5717760 5789232



100%|██████████| 81/81 [03:22<00:00,  2.50s/it]
